### Sample CNN 

Fashion Forward is a new AI-based e-commerce clothing retailer.
They want to use image classification to automatically categorize new product listings, making it easier for customers to find what they're looking for. It will also assist in inventory management by quickly sorting items.

As a data scientist tasked with implementing a garment classifier, your primary objective is to develop a machine learning model capable of accurately categorizing images of clothing items into distinct garment types such as shirts, trousers, shoes, etc.

In [70]:
!pip install torchmetrics

Defaulting to user installation because normal site-packages is not writeable


In [71]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy, Precision, Recall

In [72]:
# Load datasets
from torchvision import datasets
import torchvision.transforms as transforms

train_data = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

In [73]:
print(train_data[0][0].shape[0])
print(train_data[0][0].shape[-1])
print(train_data.targets)
len(torch.unique(train_data.targets))

1
28
tensor([9, 0, 0,  ..., 3, 0, 5])


10

In [74]:
# Start coding here
IMAGE_SIZE = train_data[0][0].shape[-1]
NUM_CLASSES = len(torch.unique(train_data.targets))
IN_CHANNELS = train_data[0][0].shape[0]



# Use as many cells as you need
class ConvNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(in_channels=IN_CHANNELS, out_channels=8, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten_ = nn.Flatten()
        self.linear = nn.Linear(8 * (IMAGE_SIZE// 2) * (IMAGE_SIZE//2), NUM_CLASSES)
        
    def forward(self, x):
        x = self.conv(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.flatten_(x)
        x = self.linear(x)
        return x

In [75]:
train_data_loader = DataLoader(train_data, batch_size=10 , shuffle=True)
test_data_loader = DataLoader(test_data, batch_size=10, shuffle=True)
model = ConvNN()

In [76]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()


def train_model(data_loader, optimizer, epochs, lr , model):
    for epoch in range(1,epochs):
        epoch_loss = 0
        num_processed = 0
        for data in data_loader:
            optimizer.zero_grad()
            features, targets = data[0], data[1]
            output = model(features)
            loss = criterion(output, targets)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            num_processed += len(targets)
        print(f"Epoch {epoch} Loss :  {epoch_loss / num_processed}")
    train_loss = epoch_loss / len(data_loader)
    print("Final Train loss:",train_loss)

In [77]:
train_model(train_data_loader, optimizer, 3, 0.001, model)

Epoch 1 Loss :  0.043536890305446774
Epoch 2 Loss :  0.032037951028265524
Final Train loss: 0.32037951028265527


In [78]:
accuracy_metric = Accuracy(task='multiclass', num_classes=NUM_CLASSES)
precision_metric = Precision(task='multiclass', num_classes=NUM_CLASSES, average=None)
recall_metric = Recall(task='multiclass', num_classes=NUM_CLASSES, average=None)
predictions = []

def eval_model(data_loader, epochs, model):
    model.eval()
    with torch.no_grad():
        for data in data_loader:
            features, targets = data[0], data[1]
            output = model(features)
            predicted = torch.argmax(output,dim=-1)
            predictions.extend(predicted)
            accuracy_metric(predicted,targets)
            precision_metric(predicted,targets)
            recall_metric(predicted,targets)

In [79]:
eval_model(test_data_loader, 3, model)

In [80]:
accuracy = accuracy_metric.compute()
precision = precision_metric.compute()
recall = recall_metric.compute()


print("Accuracy", accuracy)
print("Precision", precision)
print("Recall", recall)

Accuracy tensor(0.8833)
Precision tensor([0.8442, 0.9887, 0.7921, 0.8814, 0.7898, 0.9866, 0.6877, 0.9167, 0.9739,
        0.9742])
Recall tensor([0.8020, 0.9650, 0.8460, 0.8920, 0.8490, 0.9540, 0.6320, 0.9790, 0.9710,
        0.9430])
